# Spark Train Logistic Regression


Train Logistic Regression classifier with Apache SparkML

In [1]:
%%bash
export version=`python --version |awk '{print $2}' |awk -F"." '{print $1$2}'`

echo $version

if [ $version == '36' ] || [ $version == '37' ]; then
    echo 'Starting installation...'
    pip3 install pyspark==2.4.8 wget==3.2 pyspark2pmml==0.5.1 > install.log 2> install.log
    if [ $? == 0 ]; then
        echo 'Please <<RESTART YOUR KERNEL>> (Kernel->Restart Kernel and Clear All Outputs)'
    else
        echo 'Installation failed, please check log:'
        cat install.log
    fi
elif [ $version == '38' ] || [ $version == '39' ]; then
    pip3 install pyspark==3.1.2 wget==3.2 pyspark2pmml==0.5.1 > install.log 2> install.log
    if [ $? == 0 ]; then
        echo 'Please <<RESTART YOUR KERNEL>> (Kernel->Restart Kernel and Clear All Outputs)'
    else
        echo 'Installation failed, please check log:'
        cat install.log
    fi
else
    echo 'Currently only python 3.6, 3.7 , 3.8 and 3.9 are supported, in case you need a different version please open an issue at https://github.com/IBM/claimed/issues'
    exit -1
fi

37
Starting installation...
Please <<RESTART YOUR KERNEL>> (Kernel->Restart Kernel and Clear All Outputs)


In [2]:
from pyspark import SparkContext, SparkConf, SQLContext
import os
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark2pmml import PMMLBuilder
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler
import logging
import shutil
import site
import sys
import wget
import re

In [3]:
if sys.version[0:3] == '3.9':
    url = ('https://github.com/jpmml/jpmml-sparkml/releases/download/1.7.2/'
           'jpmml-sparkml-executable-1.7.2.jar')
    wget.download(url)
    shutil.copy('jpmml-sparkml-executable-1.7.2.jar',
                site.getsitepackages()[0] + '/pyspark/jars/')
elif sys.version[0:3] == '3.8':
    url = ('https://github.com/jpmml/jpmml-sparkml/releases/download/1.7.2/'
           'jpmml-sparkml-executable-1.7.2.jar')
    wget.download(url)
    shutil.copy('jpmml-sparkml-executable-1.7.2.jar',
                site.getsitepackages()[0] + '/pyspark/jars/')
elif sys.version[0:3] == '3.7':
    url = ('https://github.com/jpmml/jpmml-sparkml/releases/download/1.5.12/'
           'jpmml-sparkml-executable-1.5.12.jar')
    wget.download(url)
elif sys.version[0:3] == '3.6':
    url = ('https://github.com/jpmml/jpmml-sparkml/releases/download/1.5.12/'
           'jpmml-sparkml-executable-1.5.12.jar')
    wget.download(url)
else:
    raise Exception('Currently only python 3.6 , 3.7, 3,8 and 3.9 is supported, in case '
                    'you need a different version please open an issue at '
                    'https://github.com/IBM/claimed/issues')

In [4]:
data_parquet = os.environ.get('data_parquet',
                              'data.parquet')  # input file name (parquet)
master = os.environ.get('master',
                        "local[*]")  # URL to Spark master
model_target = os.environ.get('model_target',
                              "model.xml")  # model output file name
data_dir = os.environ.get('data_dir',
                          '../../data/')  # temporary directory for data
input_columns = os.environ.get('input_columns',
                               '["x", "y", "z"]')  # input columns to consider

In [5]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)

In [6]:
conf = SparkConf().setMaster(master)
#if sys.version[0:3] == '3.6' or sys.version[0:3] == '3.7':
conf.set("spark.jars", 'jpmml-sparkml-executable-1.5.12.jar')

sc = SparkContext.getOrCreate(conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession

23/03/21 21:12:29 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
df = spark.read.parquet(data_dir + data_parquet)

In [8]:
# register a corresponding query table
df.createOrReplaceTempView('df')

In [9]:
from pyspark.sql.types import DoubleType
df = df.withColumn("x", df.x.cast(DoubleType()))
df = df.withColumn("y", df.y.cast(DoubleType()))
df = df.withColumn("z", df.z.cast(DoubleType()))

In [10]:
splits = df.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

In [15]:
indexer = StringIndexer(inputCol="class", outputCol="label")

vectorAssembler = VectorAssembler(inputCols=eval(input_columns),
                                  outputCol="features")

normalizer = MinMaxScaler(inputCol="features", outputCol="features_norm")

## HyperParameter Tuning

In [23]:
def trainModelLR(maxIter=10, regParam=0.3, elasticNetParam=0.8):
    print(f'Training Model by (maxIter={maxIter}, regParam={regParam}, elasticNetParam={elasticNetParam})')
    lr = LogisticRegression(maxIter=maxIter, regParam=regParam, elasticNetParam=elasticNetParam)
    pipeline = Pipeline(stages=[indexer, vectorAssembler, normalizer, lr])
    model = pipeline.fit(df_train)
    prediction = model.transform(df_train)
    binEval = MulticlassClassificationEvaluator(). \
    setMetricName("accuracy"). \
    setPredictionCol("prediction"). \
    setLabelCol("label")
    acc = binEval.evaluate(prediction)
    print(f'The accuracy: {acc}\n')
    print('-'*100)
    return (acc, maxIter, regParam, elasticNetParam)
    

In [25]:
# Hyperparameters are training parameters
maxIters = [10, 100, 1000]
regParams = [0.01, 0.5, 2.0]
elasticNetParams = [0.0, 0.5, 1.0]
performaceList = []
for i in range(len(maxIters)):
    for j in range(len(regParams)):
        for k in range(len(elasticNetParams)):
            performaceList.append(trainModelLR(maxIters[i], regParams[j], elasticNetParams[k]))

Training Model by (maxIter=10, regParam=0.01, elasticNetParam=0.0)


The accuracy: 0.32778193665239563

----------------------------------------------------------------------------------------------------
Training Model by (maxIter=10, regParam=0.01, elasticNetParam=0.5)


The accuracy: 0.3334735201182055

----------------------------------------------------------------------------------------------------
Training Model by (maxIter=10, regParam=0.01, elasticNetParam=1.0)


The accuracy: 0.31757914244939955

----------------------------------------------------------------------------------------------------
Training Model by (maxIter=10, regParam=0.5, elasticNetParam=0.0)


The accuracy: 0.22368764141341924

----------------------------------------------------------------------------------------------------
Training Model by (maxIter=10, regParam=0.5, elasticNetParam=0.5)


...

The accuracy: 0.206697003086913

----------------------------------------------------------------------------------------------------
Training Model by (maxIter=1000, regParam=2.0, elasticNetParam=0.0)


The accuracy: 0.206697003086913

----------------------------------------------------------------------------------------------------
Training Model by (maxIter=1000, regParam=2.0, elasticNetParam=0.5)


The accuracy: 0.206697003086913

----------------------------------------------------------------------------------------------------
Training Model by (maxIter=1000, regParam=2.0, elasticNetParam=1.0)


[Stage 13075:===============================>                       (4 + 3) / 7]

The accuracy: 0.206697003086913

----------------------------------------------------------------------------------------------------


In [130]:
import numpy as np
performList = np.array(performaceList)
indexMax = performList[:,0].argmax(axis=0)
hAcc, hMaxIter, hregParam, helasti = performaceList[indexMax]
print("The highest accuracy is {} of training model by (maxIter={}, regParam={}, elasticNetParam={})".format(hAcc, hMaxIter, hregParam, helasti))

The highest accuracy is 0.35397163180221325 of training model by (maxIter=1000, regParam=0.01, elasticNetParam=1.0)


# 9. Resample data splits

70:30 train:test split \
90:10 train:test split \
Note: for both the splits above, use a random seed of 1.

In [10]:
def modelLR(maxIter=10, regParam=0.3, elasticNetParam=0.8, df_train=''):
    lr = LogisticRegression(maxIter=maxIter, regParam=regParam, elasticNetParam=elasticNetParam)
    pipeline = Pipeline(stages=[indexer, vectorAssembler, normalizer, lr])
    model = pipeline.fit(df_train)
    prediction = model.transform(df_train)
    binEval = MulticlassClassificationEvaluator(). \
    setMetricName("accuracy"). \
    setPredictionCol("prediction"). \
    setLabelCol("label")
    acc = binEval.evaluate(prediction)
    print(f'The accuracy: {acc}\n')
    print('-'*100)

In [16]:
splits70_30 = df.randomSplit([0.7, 0.3],1)
df_train70 = splits70_30[0]
df_test30 = splits70_30[1]
modelLR(hMaxIter, hregParam, helasti,df_train70)

23/03/21 21:15:38 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/03/21 21:15:38 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
[Stage 4029:================================================>       (6 + 1) / 7]

The accuracy: 0.35337080090127

----------------------------------------------------------------------------------------------------


In [17]:
splits90_10 = df.randomSplit([0.9, 0.1],1)
df_train90 = splits90_10[0]
df_test10 = splits90_10[1]
modelLR(hMaxIter, hregParam, helasti, df_train90)

[Stage 7771:================================================>       (6 + 1) / 7]

The accuracy: 0.3527924912652935

----------------------------------------------------------------------------------------------------
